# Tournament Info
- **Dataset** : 3B1-Signal
- **Description** (as on Tournament page)
    - Institutional investors are leveraging equity factor risk models (sector / country stock etc.) 
    - To predict return and hedge their bets. We investigate the extent to which nonlinearities not captured by standard linear models within equity factor risk models are present. 
    - Some generated factor returns and information ratios higher than corresponding linear factors.
-----
-----

- **Update Code**
</br></br>
    - Existing Code
        - ~~Aggregate all library imports with try-except blocks~~
        - ~~Aggregate similar category code, such as nulls-check code, into single if-else block or try-except block into single cell~~
        - ~~Add print statements in aggregated blocks of code for debugging~~
        - Aggregate Modeling section - all model initializations and model training into single cell
        - Aggregate Preditions and Submission, if possible

    - New Code
        - Save dataset locally, if needed
        - **Save trained models**
        - **Try LIGHT-GBM and XG-BOOST**
        - **Automate Workflow with MLFLOW**

-------
-------

## Import/Install Libraries

In [1]:
# Lib & Dependencies
try:
    import pandas as pd
    import numpy as np
    import xgboost as xgb
    import lightgbm as lgb
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import accuracy_score, mean_squared_error
    import requests
    from scipy import stats

except ModuleNotFoundError:
    %pip install xgboost
    %pip install lightgbm

print("\nLibraries import successful!")


Libraries import successful!


## Download & Explore data

- Training_data will be use to train your model.
- Hackathon_data will be use to make your prediciton.
- Three targets to provide predictions : target_r, target_g, target_b.

In [2]:
# Data Download (takes few minutes depending on network)
train_datalink_X = 'https://tournament.crunchdao.com/data/X_train.csv'  
train_datalink_y = 'https://tournament.crunchdao.com/data/y_train.csv' 
hackathon_data_link = 'https://tournament.crunchdao.com/data/X_test.csv' 

# Data for training
train_data = pd.read_csv(train_datalink_X)
# Data for test and submitting prediction
test_data = pd.read_csv(hackathon_data_link)
# Targets used for supervised trainning
train_targets = pd.read_csv(train_datalink_y)

print('Downloaded datasets. Created dataframes.\n')

Downloaded datasets. Created dataframes.



In [3]:
# If not working with time series
train_data = train_data.drop(columns=['id'])
test_data = test_data.drop(columns=['id'])
print('Removed column - id')

# set pandas options
pd.set_option('display.max_columns', None)

Removed column - id


In [4]:
train_data.infer_objects()

,Moons,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,Feature_10,Feature_11,Feature_12,Feature_13,Feature_14,Feature_15,Feature_16,Feature_17,Feature_18,Feature_19,Feature_20,Feature_21,Feature_22,Feature_23,Feature_24,Feature_25,Feature_26,Feature_27,Feature_28,Feature_29,Feature_30,Feature_31,Feature_32,Feature_33,Feature_34,Feature_35,Feature_36,Feature_37,Feature_38,Feature_39,Feature_40,Feature_41,Feature_42,Feature_43,Feature_44,Feature_45,Feature_46,Feature_47,Feature_48,Feature_49,Feature_50,Feature_51,Feature_52,Feature_53,Feature_54,Feature_55,Feature_56,Feature_57,Feature_58,Feature_59,Feature_60,Feature_61,Feature_62,Feature_63,Feature_64,Feature_65,Feature_66,Feature_67,Feature_68
0,0,0.5,0.25,0.75,0.50,0.00,1.00,0.25,1.00,1.00,1.00,0.50,0.00,0.00,1.00,0.75,1.00,1.00,1.00,1.00,0.00,0.25,1.00,0.0,0.50,1.00,0.50,0.5,1.00,1.00,0.75,0.75,1.00,0.50,0.00,0.50,1.00,1.00,0.75,0.00,1.00,0.00,0.75,0.50,0.50,0.75,1.00,0.25,1.00,0.00,0.50,1.00,0.25,0.75,0.5,0.0,0.0,0.0,0.5,0.5,0.50,0.5,0.75,0.00,0.25,0.50,0.0,0.5,0.00
1,0,0.0,0.50,0.25,0.75,0.25,0.25,0.25,0.50,0.50,0.50,0.75,0.25,0.75,0.25,0.75,0.50,0.75,0.50,0.75,0.50,0.25,0.00,0.5,1.00,0.50,0.00,0.0,1.00,0.75,0.25,0.25,0.50,0.50,0.00,0.75,0.50,0.75,0.50,0.75,0.25,0.25,0.50,0.25,0.25,0.75,0.50,0.75,0.50,0.00,0.50,0.50,0.50,0.50,0.5,0.0,0.5,0.5,0.5,0.5,0.50,0.5,0.00,0.25,0.00,0.75,0.0,0.0,0.50
2,0,0.5,0.00,0.00,0.00,0.75,0.00,0.00,0.00,1.00,1.00,0.25,0.75,0.00,0.00,0.25,0.00,0.50,0.25,0.00,0.25,0.00,0.75,0.0,0.00,0.25,0.50,0.0,1.00,0.00,0.00,0.75,0.50,0.00,0.75,1.00,0.50,0.25,0.00,0.75,0.25,0.25,0.25,1.00,0.75,0.25,0.25,0.50,0.25,1.00,0.75,0.50,0.25,0.75,0.0,0.0,0.0,0.5,0.0,0.5,0.25,0.0,0.00,1.00,0.50,0.00,0.0,0.0,0.50
3,0,0.0,0.75,0.00,0.75,1.00,0.50,0.25,1.00,0.75,0.75,0.75,0.50,0.50,0.50,0.50,0.75,0.50,0.00,0.50,1.00,0.25,1.00,0.5,1.00,0.75,0.50,0.5,0.50,0.50,0.75,0.75,0.50,1.00,0.25,0.00,0.00,0.50,1.00,1.00,0.00,0.75,1.00,1.00,1.00,1.00,0.75,0.50,0.75,1.00,0.25,0.75,1.00,0.75,0.0,0.0,0.0,0.0,0.0,0.5,0.50,0.5,0.50,1.00,1.00,0.00,0.0,0.0,0.75
4,0,0.5,0.75,0.75,0.25,0.50,1.00,1.00,1.00,0.25,0.25,0.50,1.00,0.50,1.00,0.50,0.75,0.25,0.25,0.75,0.25,0.25,0.00,0.0,0.50,0.00,1.00,0.5,0.75,1.00,0.50,0.25,0.25,0.75,0.75,0.25,0.00,0.25,1.00,0.50,0.25,0.75,1.00,0.75,0.50,0.25,0.25,0.00,0.25,0.25,0.75,0.00,0.25,0.00,0.0,0.0,0.0,0.5,0.5,0.5,0.25,0.0,0.50,0.75,0.25,0.75,0.0,0.0,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
520989,448,1.0,0.00,0.50,0.25,0.75,0.00,0.25,1.00,0.25,0.25,0.75,0.25,0.25,0.00,0.00,0.25,0.00,0.75,0.25,0.25,1.00,0.50,1.0,0.75,0.00,0.00,0.5,1.00,0.75,0.00,0.25,0.00,0.25,1.00,0.00,0.25,0.00,0.25,1.00,1.00,0.25,0.00,0.50,1.00,0.00,0.25,0.25,0.25,0.50,1.00,0.50,0.00,0.00,1.0,1.0,0.5,0.5,0.5,0.5,0.00,0.5,0.00,0.75,0.00,0.75,1.0,0.5,1.00
520990,448,1.0,0.00,0.00,0.25,1.00,0.50,0.00,0.00,0.00,0.00,0.00,1.00,1.00,0.50,1.00,1.00,0.75,0.00,0.50,0.75,1.00,1.00,0.0,0.00,0.00,0.75,0.5,0.50,0.75,0.50,0.00,0.00,0.50,1.00,1.00,0.25,0.00,0.00,0.25,0.00,0.00,1.00,1.00,1.00,0.25,0.00,0.00,0.00,0.25,0.50,0.25,0.00,0.25,1.0,1.0,0.5,0.5,1.0,1.0,1.00,1.0,0.25,1.00,1.00,0.00,1.0,1.0,1.00
520991,448,0.0,0.50,1.00,1.00,0.00,1.00,1.00,0.75,0.75,0.75,0.75,0.25,1.00,1.00,0.50,1.00,0.75,0.25,0.75,0.50,1.00,1.00,1.0,0.25,0.50,0.50,1.0,0.75,0.25,1.00,1.00,0.75,0.50,0.25,0.25,1.00,1.00,1.00,0.00,0.25,1.00,1.00,0.00,1.00,1.00,0.75,1.00,0.75,0.00,0.75,0.25,0.75,0.25,1.0,1.0,1.0,1.0,1.0,1.0,1.00,1.0,0.25,0.75,0.75,1.00,1.0,1.0,0.25
520992,448,1.0,1.00,0.25,1.00,0.00,0.50,1.00,0.75,1.00,1.00,0.75,1.00,0.75,0.50,1.00,1.00,0.75,1.00,0.75,1.00,1.00,1.00,1.0,1.00,0.75,0.75,1.0,1.00,0.25,0.75,1.00,1.00,1.00,0.25,0.00,0.75,1.00,0.50,0.50,0.75,0.50,1.00,1.00,0.00,1.00,1.00,1.00,1.00,0.00,0.00,1.00,0.75,0.75,1.0,1.0,0.5,1.0,0.5,1.0,

In [5]:
test_data.infer_objects()

,Moons,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,Feature_10,Feature_11,Feature_12,Feature_13,Feature_14,Feature_15,Feature_16,Feature_17,Feature_18,Feature_19,Feature_20,Feature_21,Feature_22,Feature_23,Feature_24,Feature_25,Feature_26,Feature_27,Feature_28,Feature_29,Feature_30,Feature_31,Feature_32,Feature_33,Feature_34,Feature_35,Feature_36,Feature_37,Feature_38,Feature_39,Feature_40,Feature_41,Feature_42,Feature_43,Feature_44,Feature_45,Feature_46,Feature_47,Feature_48,Feature_49,Feature_50,Feature_51,Feature_52,Feature_53,Feature_54,Feature_55,Feature_56,Feature_57,Feature_58,Feature_59,Feature_60,Feature_61,Feature_62,Feature_63,Feature_64,Feature_65,Feature_66,Feature_67,Feature_68
0,0,0.25,1.00,0.25,1.00,0.25,0.00,0.00,0.75,0.50,0.25,0.75,0.25,1.00,0.00,0.75,0.00,0.25,0.25,0.00,1.00,0.00,0.75,0.00,0.75,0.50,0.50,0.0,0.75,0.50,0.25,1.00,0.50,0.25,0.00,0.25,0.75,1.00,0.00,0.50,1.00,0.75,0.50,0.25,0.0,0.75,0.25,0.25,0.25,0.75,0.50,0.25,0.75,0.75,0.5,0.0,0.5,0.75,0.0,0.0,0.75,0.0,0.75,0.25,0.50,0.75,0.0,0.0,0.75
1,0,0.00,1.00,0.00,1.00,0.00,0.00,1.00,1.00,1.00,1.00,0.75,1.00,0.75,0.00,0.50,0.75,1.00,1.00,0.75,0.25,0.00,1.00,0.00,0.00,1.00,0.25,0.0,1.00,0.00,0.50,1.00,1.00,0.50,0.00,0.50,0.75,1.00,0.25,0.00,0.75,0.75,0.50,1.00,0.0,1.00,1.00,0.00,1.00,0.00,1.00,0.75,1.00,0.50,0.0,0.0,0.5,0.00,0.0,0.0,0.75,0.5,0.00,0.00,0.00,0.75,0.0,0.5,0.00
2,0,1.00,1.00,0.00,0.00,1.00,0.75,0.00,0.00,0.00,0.00,0.00,0.75,1.00,1.00,1.00,1.00,0.50,0.00,0.00,0.25,0.00,0.75,0.00,0.50,0.00,1.00,0.0,0.25,0.25,0.00,0.00,0.00,0.25,1.00,0.50,0.00,0.00,0.50,0.75,0.00,0.25,1.00,0.50,0.5,0.00,0.00,0.25,0.00,0.75,0.25,0.00,0.75,0.50,0.0,0.0,0.0,0.00,0.0,0.5,0.75,0.5,1.00,1.00,0.75,0.25,0.0,0.0,1.00
3,0,0.25,0.75,0.50,0.75,0.50,0.25,0.75,0.75,0.50,0.25,1.00,0.75,0.25,0.25,0.50,0.75,0.00,0.50,0.25,0.50,0.00,0.25,0.00,0.75,0.25,0.25,0.0,0.75,0.75,0.25,0.25,0.25,0.00,0.00,0.75,0.00,0.75,0.50,0.25,0.00,0.75,0.50,1.00,0.0,0.50,0.25,0.50,0.25,0.50,0.25,0.25,0.50,0.00,0.0,0.0,0.5,0.75,0.0,0.0,0.00,0.0,0.25,0.25,0.75,0.75,0.0,0.0,0.50
4,0,1.00,0.00,0.00,0.25,1.00,0.00,0.75,0.25,0.00,0.00,0.75,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.50,0.50,0.00,0.25,0.00,0.00,1.00,0.00,0.25,0.00,0.00,1.00,0.00,0.75,0.00,0.00,0.5,0.25,0.00,0.00,0.00,0.25,0.25,0.00,0.00,0.00,0.0,0.0,0.5,0.00,0.0,0.0,0.00,0.0,0.00,1.00,0.25,0.00,0.0,0.0,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16154,12,0.25,0.50,0.75,0.75,0.00,1.00,0.75,0.00,0.75,0.75,0.00,0.00,0.00,1.00,1.00,1.00,1.00,0.25,1.00,0.25,1.00,0.25,1.00,1.00,1.00,1.00,1.0,1.00,0.00,1.00,0.75,1.00,1.00,0.00,1.00,0.75,1.00,1.00,0.00,0.75,0.00,1.00,0.50,1.0,0.50,0.75,1.00,0.75,0.75,0.75,0.25,0.75,0.75,0.5,1.0,0.5,0.50,1.0,0.5,0.25,0.5,0.75,0.00,0.50,0.25,1.0,1.0,0.25
16155,12,0.75,0.00,0.00,0.00,1.00,0.00,0.00,0.25,0.25,0.25,1.00,0.50,0.25,0.00,0.25,0.25,0.00,0.75,0.25,0.50,0.25,0.50,0.50,0.75,0.50,0.00,0.5,0.25,0.50,0.50,0.25,0.50,0.00,0.75,0.25,0.25,0.25,0.50,0.25,1.00,1.00,0.00,0.50,1.0,0.00,0.25,0.00,0.25,0.50,0.00,0.25,0.25,1.00,0.5,1.0,0.5,0.50,0.5,0.5,0.00,0.5,1.00,0.50,0.00,0.75,1.0,0.5,0.50
16156,12,0.00,0.50,0.50,0.75,0.25,0.50,0.75,1.00,0.50,0.50,1.00,0.25,1.00,0.25,1.00,1.00,0.50,0.50,0.50,0.50,1.00,0.50,1.00,0.50,0.75,0.00,1.0,0.25,0.75,1.00,1.00,0.50,0.50,0.25,0.25,1.00,0.75,0.50,0.75,0.75,1.00,0.50,0.00,0.0,0.75,0.50,0.25,0.50,0.50,0.75,0.50,0.75,0.75,1.0,1.0,1.0,0.50,1.0,0.5,0.50,0.5,0.75,0.00,0.25,1.00,1.0,0.5,0.50
16157,12,0.00,0.50,0.00,1.00,0.00,0.00,0.75,0.75,0.75,0.75,0.25,1.00,0.00,0.00,0.00,0.00,0.50,0.50,0.50,0.75,1.00,0.00,1.00,0.50,0.75,0.00,0.5,0.25,0.25,0.00,0.75,0.50,0.75,0.00,0.00,0.50,0.75,0.00,0.75,0.50,0.75,0.75,0.75,0.5,1.00,0.75,0.00,0.75,0.25,0.25,0.75,0.75,0.25,1.0,1.0,1.0,1.0

In [6]:
train_targets.infer_objects()

,target_r,target_g,target_b
0,0.00,0.0,0.0
1,0.25,0.5,1.0
2,0.00,0.0,0.0
3,1.00,1.0,1.0
4,0.00,0.0,0.0
...,...,...,...
520989,0.25,0.5,0.5
520990,1.00,0.0,0.0
520991,0.50,0.0,0.0
520992,0.00,0.0,0.5


## Modeling

- **Models explored so far :**
    - Random Forest Regressor --> too slow on large datasets (1M+ records) - generally fast (< 500K+ records)
    - Extra Trees Regressor --> no enhanced effect on any dataset so far - moderately fast
    - XG Boost --> Succesful candidate so far - slowest so far
    - Voting Regressor --> Helps in finalizing on choosing model.
</br></br>
- **Models exploring now :**
    - Light GBM
    - XG Boost
    - Voting Regressor
</br></br>
- **Models to explore :**
    - [N-beats keras/pytorch](https://github.com/philipperemy/n-beats)

- Extra Trees Regressor

In [8]:
# Extra Tress Regressor Initialization

# seed for reproducibility
import random
SEED = random.seed(41)

from sklearn.ensemble import ExtraTreesRegressor

et_reg = ExtraTreesRegressor(n_estimators=1000, max_depth=5, min_samples_split=100, min_samples_leaf=50, min_weight_fraction_leaf=0.4, ccp_alpha=0.5, max_samples=0.8,
                            bootstrap=True, n_jobs=-1, random_state=SEED)



- **XGB Parameters** --> too long for 500k + records???

- Source : [XG Boost Documentation](https://xgboost.readthedocs.io/en/stable/)
    - Additional XGBoostRegressor Parameters (tree_method='gpu_hist')
    - Need to Figure out -- LEARNING TASK PARAMETERS
    - Increase test_size to 0.2

In [14]:
# XG Boost Regressor Initialization
xgb_reg = xgb.XGBRegressor(max_depth=10, min_child_weight=4, subsample=0.5, learning_rate=0.2, n_estimators=20000, colsample_bytree=0.5, colsample_bylevel=0.5, colsample_bynode=0.5,
                            tree_method="gpu_hist", num_parallel_tree=5, 
                            objective='reg:squaredlogerror', eval_metric='rmsle')
# sample_type='weighted', normalize_type='forest', feature_selector='thrifty', top_k=25



- **Light GBM Parameters**
- Source : [Light GBM Documentation](https://lightgbm.readthedocs.io/en/latest/Python-Intro.html)
    - Read Gradient Boosting guide by [MachineLearningMastery](https://machinelearningmastery.com/gradient-boosting-with-scikit-learn-xgboost-lightgbm-and-catboost/)
        - includes sklearn, XGboost, lightGBM, catboost

In [9]:
# Light GBM Regressor Initialization

# define validation data
rows = random.randint(10000, 50000)
validation_data = train_data.iloc[:rows, :]

# define parameters
#param = {'objective':'regression','n_estimators':100000,'num_leaves': 500,'early_stopping_round':100,'device':'gpu','learning_rate':0.25,'max_leaves':1000,
#        'num_iterations':10,'device_type':'gpu','tree_learner':'voting','top_k':70,'seed':SEED,'metric':'rmse'}

lgb_reg = lgb.LGBMRegressor(boosting_type='gbdt', device_type='gpu', n_estimators=10000, num_leaves=500) #!!! In secs for 500K+ records


- **Scoring, Voting & Training**

In [10]:
# Scoring calculation for trained models
def scorer(y_test, y_pred):

    score = (stats.spearmanr(y_test, y_pred)*100)[0]
    print('Score as calculated for the leader board (っಠ‿ಠ)っ {}'.format(score))

In [11]:
# Voting Regressor Initialization
from sklearn.ensemble import VotingRegressor

models = [('lgb', lgb_reg), ('et', et_reg)]
vote_reg = VotingRegressor(estimators=models, verbose=True)

def best_reg(data, target):
    X, y = data, target
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, shuffle=True, random_state=SEED)    
    vote_reg.fit(X_train, y_train)
    preds = vote_reg.predict(X_test)
    scorer(y_test, preds)

    return vote_reg

In [12]:
# Train, Test and Score Models
model_target_r = best_reg(train_data, train_targets.target_r)
print('Trained target_r models.')
model_target_g = best_reg(train_data, train_targets.target_g)
print('Trained target_g models.')
model_target_b = best_reg(train_data, train_targets.target_b)
print('Trained target_b models.')

[Voting] ...................... (1 of 2) Processing lgb, total=30.3min
[Voting] ....................... (2 of 2) Processing et, total= 1.7min
Score as calculated for the leader board (っಠ‿ಠ)っ 0.09676970595622958
Trained target_r models.
[Voting] ...................... (1 of 2) Processing lgb, total=31.3min
[Voting] ....................... (2 of 2) Processing et, total= 1.7min
Score as calculated for the leader board (っಠ‿ಠ)っ 0.06735161133656205
Trained target_g models.
[Voting] ...................... (1 of 2) Processing lgb, total=32.1min
[Voting] ....................... (2 of 2) Processing et, total= 1.7min
Score as calculated for the leader board (っಠ‿ಠ)っ 0.07014197868879324
Trained target_b models.


## Predictions

* When model is accurate enough it's time to predict the target and submit results.
* Repeat operation on the three targets, concatenate the answers and submit.

- MUST Do's :
    1. **Keep raw order identical.**
    2. **Be sure that columns are named as : target_r, target_g and target_b.**
    3. **Prediction need to be between 0 and 1.**   
    4. **Don't submit constant values.**

In [13]:
prediction = pd.DataFrame()
prediction['target_r'] = model_target_r.predict(test_data)
prediction['target_g'] = model_target_g.predict(test_data)
prediction['target_b'] = model_target_b.predict(test_data)

In [14]:
prediction

,target_r,target_g,target_b
0,0.449826,0.449826,0.449826
1,0.427171,0.427171,0.427171
2,0.579135,0.579135,0.579135
3,0.539090,0.539090,0.539090
4,0.430045,0.430045,0.430045
...,...,...,...
16154,0.618900,0.618900,0.618900
16155,0.537149,0.537149,0.537149
16156,0.523160,0.523160,0.523160
16157,0.507041,0.507041,0.507041


### Submission
* Import API_KEY from .env
* Possible error codes and their descriptions

In [15]:
# Import libraries required for submission

#%pip install python-dotenv 
import os
from dotenv import load_dotenv
import requests


In [16]:

# Import API_KEY from .env

load_dotenv('D:\GitHub_Projects\crunchdao\.env')

API_KEY =  os.getenv('SECRET_KEY')

if API_KEY != None:
    print('Key Import : Nailed It!')
else:
    print('Key Missing!')

Key Import : Nailed It!


In [17]:
# Push Predictions

r = requests.post("https://tournament.crunchdao.com/api/v2/submissions",
    files = {
        "file": ("x", prediction.to_csv().encode('ascii'))
    },
    data = {
        "apiKey": API_KEY
    },
)

if r.status_code == 200:
    print("Submission submitted.")
elif r.status_code == 423:
    print("ERR: Submissions are close")
    print("You can only submit during rounds eg: Friday 7pm GMT+1 to Sunday midnight GMT+1.")
    print("Or the server is currently crunching the submitted files, please wait some time before retrying.")
elif r.status_code == 422:
    print("ERR: API Key is missing or empty")
    print("Did you forget to fill the API_KEY variable?")
elif r.status_code == 400:
    print("ERR: The file must not be empty")
    print("You have send a empty file.")
elif r.status_code == 401:
    print("ERR: Your email hasn't been verified")
    print("Please verify your email or contact a cruncher.")
elif r.status_code == 409:
    print("ERR: Duplicate submission")
    print("Your work has already been submitted with the same exact results, if you think that this a false positive, contact a cruncher.")
    print("MD5 collision probability: 1/2^128 (source: https://stackoverflow.com/a/288519/7292958)")
elif r.status_code == 429:
    print("ERR: Too many submissions")
else:
    print("ERR: Server returned: " + str(r.status_code))
    print("Ouch! It seems that we were not expecting this kind of result from the server, if the probleme persist, contact a cruncher.")

Submission submitted.
